In [ ]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
import pandas as pd
import sys

sys.path.append('/sbnd/app/users/brindenc/mysbnana_v09_75_03/srcs/sbnana/sbnana/SBNAna/pyana')
from makedf import geniesyst
from sbnd.cafclasses.nu import NU
from sbnd.general import utils

%load_ext autoreload
%autoreload 2

: 

## Testing

## Testing Genie

In [8]:
fname = '/pnfs/sbn/data/sbn_nd/poms_production/official/MCP2022A/v09_37_02_04/prodoverlay_corsika_cosmics_proton_genie_rockbox_sce/reco2_caf/bc/be/reco2.flat.caf-e09ac695-01b7-44d7-9ef7-0172fb1bca3c.root'
tree = uproot.open(fname)['recTree;3']
keys = [
  'rec.mc.nu.genweight',
]
df = tree.arrays(keys, library='pd')
nu = NU(df)

In [9]:
nu.keys()

Index(['rec.mc.nu.genweight'], dtype='object')

## Testing Numi

In [13]:
FSYST = "/cvmfs/sbn.opensciencegrid.org/products/sbn/sbndata/v01_04/beamData/NuMIdata/icarus_numi_flux_syst_ana.root"

In [19]:
beam_uncertainties = [
    "beam_div",
    "beam_shift_x",
    "beam_spot",
    "horn1_x",
    "horn1_y",
    "horn_current_plus",
    "water_layer"
]

pca_components = list(range(20))

def histdf(h):
    values = h.values()
    bins = h.axis().edges()
    idx = pd.IntervalIndex.from_breaks(bins)
    return pd.Series(values, idx, name=None)
           
def getallpdg_histdf(d, prefix):
    pdgs = {
        "numu": 14,
        "numubar": -14,
        "nue": 12,
        "nuebar": -12   
    }
    hs = []
    for pdgname, pdgcode in pdgs.items():
        h = histdf(d[prefix + pdgname])
        h.index = pd.MultiIndex.from_product([[pdgcode], h.index], names=["pdg", "E"])
        hs.append(h)
    return pd.concat(hs)
def numisyst(nupdg, nuE, fsyst=FSYST):
    flux_f = uproot.open(fsyst)

    cv = getallpdg_histdf(flux_f["ppfx_output"]["fhc"]["nom"], "hcv_") / getallpdg_histdf(flux_f["ppfx_output"]["fhc"]["nom"], "hnom_")
    cv.name = ("ppfx", "cv")

    beam_syst_wgts = []
    for uc in beam_uncertainties:
        uncdf = getallpdg_histdf(flux_f["fractional_uncertainties"]["beam"][uc], "hfrac_beam_" + uc + "_fhc_")
        wgtdf_p = 1 + uncdf
        wgtdf_m = 1 - uncdf
        wgtdf_p.name = (uc, "ps")
        wgtdf_m.name = (uc, "ms")
        beam_syst_wgts.append(wgtdf_p)
        beam_syst_wgts.append(wgtdf_m)
    
    # for i in pca_components:
    #     uncdf = getallpdg_histdf(flux_f["pca"]["principal_components"], "hpc_%i_fhc_" % i)
    #     wgtdf_p = 1 + uncdf
    #     wgtdf_m = 1 - uncdf
    #     wgtdf_p.name = (("pca%i" % i), "ps")
    #     wgtdf_m.name = (("pca%i" % i), "ms")
    #     beam_syst_wgts.append(wgtdf_p)
    #     beam_syst_wgts.append(wgtdf_m)

    wgts = pd.DataFrame([cv] + beam_syst_wgts).T
    print(nupdg,nuE)
    nuind = pd.MultiIndex.from_arrays([nupdg, nuE])
    iloc = wgts.index.get_indexer(nuind)
    match_wgts = wgts.iloc[iloc]
    match_wgts.loc[iloc < 0, :] = 1.
    match_wgts.index = nuE.index

    return match_wgts

In [20]:
cv = getallpdg_histdf(flux_f["ppfx_output"]["fhc"]["nom"], "hcv_") / getallpdg_histdf(flux_f["ppfx_output"]["fhc"]["nom"], "hnom_")
cv.name = ("ppfx", "cv")

In [22]:

numisyst([14,12],[1,1])

[14, 12] [1, 1]


/sbnd/data/users/brindenc/.local/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


TypeError: 'builtin_function_or_method' object is not iterable